In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from utils import *

Based on Lemma 4.3, the following code computes the interval which the value of $\log(N)$ cannot belong to, and then provides upper bounds for $\log(N)$.

Here $N = |abc|/\gcd(16,abc)$, thus $\log(|abc|) \le \log(N)+\log(16)$, hence we can also get upper bounds for $\log(abc)$.

In [2]:
e0 = 3

def get_ramification_dataset1(l):
    # the case of general Frey-Hellegouarch curves
    ep_range = dict()
    ep_range['general'] = {'good':[1], 'multi':[1,3,l,3*l]} # general ramification indices, e0 = 3
    ep_range[2] = {'good':[2,4,6,8,12,16,24,48], 'multi':[2,6,2*l,6*l]}
    ep_range[3] = {'good':[2,6,8], 'multi':[2,6,2*l,6*l]}
    ep_range[l] = {'good':[l-1,l*(l-1),l*l-1], 'multi':[l-1,3*(l-1),l*(l-1),3*l*(l-1)]}
    return ep_range

def get_ramification_dataset2(l):
    # the case of semi-stable Frey-Hellegouarch curves
    ep_range = dict()
    ep_range['general'] = {'good':[1], 'multi':[1,3,l,3*l]} # general ramification indices, e0 = 3
    ep_range[2] = {'good':[2], 'multi':[2,6,2*l,6*l]}
    ep_range[3] = {'good':[2,6,8], 'multi':[2,6,2*l,6*l]}
    ep_range[l] = {'good':[l-1,l*(l-1),l*l-1], 'multi':[l-1,3*(l-1),l*(l-1),3*l*(l-1)]}
    return ep_range

In [ ]:
# compute log-volumes
list_of_f2_1e4 = list_of_f2(pow(10,5)) # recall that f2 is an upper bound of log-volume

# case 1
my_log_volumes1 = dict()
for p in tqdm(primes(pow(10,5))):
    if p >= 5 and p <= 50:
        my_log_volumes1[p] = compute_log_volume(get_ramification_dataset1, e0, p)
    elif p > 50:
        my_log_volumes1[p] = list_of_f2_1e4[p]

# case 2
my_log_volumes2 = dict()
for p in tqdm(primes(pow(10,5))):
    if p >= 5 and p <= 50:
        my_log_volumes2[p] = compute_log_volume(get_ramification_dataset2, e0, p)
    elif p > 50:
        my_log_volumes2[p] = list_of_f2_1e4[p]

We shall assume that $s\ge r$.

In [4]:
def bound_of_log_N(r, t, is_using_t_eq_r = False, s = 0):
    is_semistable_over_Q = min(r,t) >= 4
    
    if is_semistable_over_Q:
        my_primes = [p for p in primes(2000) if p >= 11]
        my_log_volumes = my_log_volumes2.copy()
    else:
        my_primes = [p for p in primes(2000) if p >= 11 and p != 13]
        my_log_volumes = my_log_volumes1.copy()
    
    intervals = []
    arr_S = []
    for n in range(2,20):
        for k in range(len(my_primes)-n):
            arr_S.append(my_primes[k:k+n])
    for k in range(len(arr_S)):
        S = sorted(list(arr_S[k]))
        n = len(S)
        a1 = 0
        a2 = 0
        a3 = 0
        for p in S:
            a1 += (11*p+31) / (p*p+p-12)
            a2 += my_log_volumes[p]  
            a3 += log(p)
        a1 = a1 / n
        a2 = 3 * a2 / n
        a3 = a3

        p0 = S[0]
        if r >= t:
            b1 = max(2/n+(3+a1)/p0, ((3+a1)/t + 2/n+(3+a1)/p0)/2, (3+a1)*(1/r+1/t)/2 )
        else: # r < t
            b1 = max(2/n+(3+a1)/p0, (3+a1)/r * 2/3 + (2/n+(3+a1)/p0)/3, (3+a1)*(1/r * 2/3 + 1/t / 3) )

        # is_using_t_eq_r
        if r == t and is_using_t_eq_r:
            b1 = max(2/n+(3+a1)/p0, (2/n+(3+a1)/p0)*(t-1)/(3*t-1) + (3+a1)*2/(3*t-1), (3+a1)*(2+(t-1)/s)/(3*t-1))
        b2 = a2 + (3+a1)*(a3+log(16)/min(r,t))

        if b1>=1:
            continue
        upper_bound = p0*S[1]*log(2)
        lower_bound = b2 / (1-b1)
        if lower_bound > upper_bound:
            continue
        intervals.append([lower_bound,upper_bound])

    res = merge_intervals(intervals)
    return res

def bound_of_log_abc(r, t, is_using_t_eq_r = False, s = 0):
    intervals = bound_of_log_N(r, t, is_using_t_eq_r, s)
    if len(intervals) == 0:
        print("Error: no impossible interval")
        return -999
    if intervals[-1][1] <= 10^6:
        # check that the interval of impossible values of \log(N) covers 10^6 
        print("Error: existing possible \log(N) near 10^6")
    return intervals[-1][0] + log(16)    

In [ ]:
# upper bound B(r,t) for given (r,t), where r <= s and (r,t) != (3,3)
# by the definition of b1 in the above code, we can see that: 
# when r <= t < t1, we have B(r,t) <= B(r,t1); when t <= r < t1, we have B(r,t) <= B(r1,t) 
for r in range(3,10):
    for t in range(3,10):
        if r == 3 and t == 3:
            print(f'({r},{t}): NaN, ', end='')
            continue
        print(f'({r},{t}): {ceil(bound_of_log_abc(r,t)+0.1):4d}, ', end='')
    print()

In [ ]:
r,t = 3,3
for s in range(4,10):
    print(f'({r},{s},{t}): {ceil(bound_of_log_abc(r,t,True,s)+0.1)}')

Hence when $(\min\{r,s\},t) = (3,3)$, we have $f(r,s,t) \le 24626$.